In [3]:
import nltk
import textstat as tx
import numpy as np
import xlsxwriter
import pandas as pd
import PyPDF2
import re

In [ ]:
# Save the standard as a pdf and put it in the same directory as the notebook. Change the name from test.pdf to standardName.pdf
# e.g, ASMEY14.pdf
file = 'test.pdf'

In [4]:
# This will split the pdf into sentences 
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mpb1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Creates functions for readability scores 
# Documentation for the different scores can be found here: https://pypi.org/project/textstat/ 
def readability(df):
   return df['sentence'].apply(tx.flesch_kincaid_grade)
def readability2(df):
    return df['sentence'].apply(tx.syllable_count)
def readability3(df):
    return df['sentence'].apply(tx.lexicon_count)
def readability4(df):
    return df['sentence'].apply(tx.automated_readability_index)
def readability5(df):
    return df['sentence'].apply(tx.gunning_fog)
def readability6(df):
    return df['sentence'].apply(tx.text_standard)

In [6]:
# Function to convert pdf into a text document for analysis 
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [7]:
# This converts the pdf to text
text = convert_pdf_to_txt(file)
# This removes any special characters \n and makes them spaces. 
text = text.replace('\n','')

In [136]:
# Creates a list of the sentences from the standard 
# Note this will not break it into each sentence, normally will be chunks of sentences. 
a_list = nltk.tokenize.sent_tokenize(text)

In [138]:
# This creates a dataframe with the sentences and scores 
df2 = pd.DataFrame(data={'sentence': a_list}).assign(overall_score=readability6).assign(num_words=readability3).assign(num_syllab=readability2).assign(FK_score=readability).assign(ari_score=readability4).assign(fog_score=readability5)

In [139]:
# This will print the dataframe so you can check 
df2

,sentence,overall_score,num_words,num_syllab,FK_score,ari_score,fog_score
0,INTERNATIONAL STANDARDISO20140-1First edition2...,-1th and 0th grade,53,123,32.2,41.5,35.54
1,ANSI order DA_330766.,9th and 10th grade,3,5,5.6,9.9,1.20
2,Downloaded 8/7/2013 2:24 PM.,7th and 8th grade,4,6,3.7,10.0,11.60
3,Single user license only.,11th and 12th grade,4,8,9.6,6.5,1.60
4,Copying and networking prohibited.© ISO 2013Re...,13th and 14th grade,17,33,13.5,26.7,18.56
...,...,...,...,...,...,...,...
368,ISO 20140-1:2013(E) ICS 25.040.01Price based o...,4th and 5th grade,18,20,4.4,13.8,9.42
369,ANSI order DA_330766.,9th and 10th grade,3,5,5.6,9.9,1.20
370,Downloaded 8/7/2013 2:24 PM.,7th and 8th grade,4,6,3.7,10.0,11.60
371,Single user license only.,11th and 12th grade,4,8,9.6,6.5,1.60


In [141]:
# This will save the outputted dataframe to an excel file called 'output.xlsx'
with pd.ExcelWriter('output.xlsx') as writer: 
    df2.to_excel(writer, sheet_name = 'test')

In [4]:
# This will allow you to enter a new sentence to check the scores. It only checks Flesch scores for simplicity. 
sent_input = input("Enter the sentence you want to check : ") 
print("The Flesch Reading Grade score: ",tx.flesch_kincaid_grade(sent_input))
print("The Flesch Reading Ease score: ",tx.flesch_reading_ease(sent_input))

Enter the sentence you want to check : This is a test sentence. 
The Flesch Reading Grade score:  2.9
The Flesch Reading Ease score:  83.32
